In [1]:
import os
import pickle as pkl
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
import pydeseq2
from pydeseq2.dds import DeseqDataSet
from pydeseq2.default_inference import DefaultInference
from pydeseq2.ds import DeseqStats

In [5]:
from pydeseq2.utils import build_design_matrix
from deconveil.dds import deconveil_fit
from deconveil.inference import Inference
from deconveil.default_inference import DefInference
from deconveil.utils_CNaware import *
from deconveil import deconveil_fit
from deconveil.ds import deconveil_stats

In [7]:
def run_pydeseq2(rna_counts, metadata, output_path, design_factors="condition", alpha=0.05):
    """
    Runs PyDESeq2 analysis and saves the results.

    Parameters:
        rna_counts (pd.DataFrame): Count matrix with genes as rows and samples as columns.
        metadata (pd.DataFrame): Metadata for the samples with design factors.
        output_path (str): Directory to save the results.
        design_factors (str): Column in metadata to use for design.
        alpha (float): Significance level for statistical tests.
    """
    os.makedirs(output_path, exist_ok=True)
    
    # Initialize DESeq2 analysis
    inference = DefaultInference(n_cpus=8)
    dds = DeseqDataSet(
        counts=rna_counts,
        metadata=metadata,
        design_factors="condition",
        refit_cooks=True,
        inference=inference,
    )

    # Fit DESeq2 model
    dds.fit_size_factors()
    dds.fit_genewise_dispersions()
    dds.fit_dispersion_trend()
    dds.fit_dispersion_prior()
    dds.fit_MAP_dispersions()
    dds.fit_LFC()
    dds.calculate_cooks()
    
    if dds.refit_cooks:
        dds.refit()

    # Perform statistical analysis
    stat_res_pydeseq = DeseqStats(dds, alpha=alpha, cooks_filter=True, independent_filter=True)
    stat_res_pydeseq.run_wald_test()

    if stat_res_pydeseq.cooks_filter:
        stat_res_pydeseq._cooks_filtering()
    stat_res_pydeseq.p_values

    if stat_res_pydeseq.independent_filter:
        stat_res_pydeseq._independent_filtering()
    else:
        stat_res_pydeseq._p_value_adjustment()

    # Log-fold change shrinkage
    stat_res_pydeseq.lfc_shrink(coeff="condition_B_vs_A")
    stat_res_pydeseq.summary()

    # Save results
    results_path = os.path.join(output_path, f"{replicate_num}_res_CNnaive_{n_samples}_{n_genes}.csv")
    stat_res_pydeseq.results_df.to_csv(results_path)
    return(stat_res_pydeseq.results_df)


def run_deconveil(rna_counts, metadata, cnv, output_path, design_factors="condition", alpha=0.05):
    """
    Runs DeConveil analysis and saves the results.

    Parameters:
        rna_counts (pd.DataFrame): Count matrix with genes as rows and samples as columns.
        metadata (pd.DataFrame): Metadata for the samples with design factors.
        cnv (pd.DataFrame): Copy number variation (CNV) data matrix  with genes as rows and samples as columns.
        output_path (str): Directory to save the results.
        design_factors (str): Column in metadata to use for design.
        alpha (float): Significance level for statistical tests.
    """
    os.makedirs(output_path, exist_ok=True)
    
    # Initialize DeConveil inference
    inference = DefInference(n_cpus=8)

    # Fit DeConveil model
    dds = deconveil_fit(
        counts=rna_counts,
        metadata=metadata,
        cnv=cnv,
        design_factors=design_factors,
        inference=inference,
        refit_cooks=True
    )
    dds.fit_size_factors()
    dds.fit_genewise_dispersions()
    dds.fit_dispersion_trend()
    dds.fit_dispersion_prior()
    dds.fit_MAP_dispersions()
    dds.fit_LFC()
    dds.calculate_cooks()

    if dds.refit_cooks:
        dds.refit()  # Replace outlier counts

    # Statistical analysis
    stat_res_deconveil = deconveil_stats(
        dds, 
        alpha=alpha, 
        independent_filter=True, 
        cooks_filter=True
    )
    stat_res_deconveil.run_wald_test()

    if stat_res_deconveil.independent_filter:
        stat_res_deconveil._independent_filtering()
    else:
        stat_res_deconveil._p_value_adjustment()

    # Log-fold change shrinkage
    stat_res_deconveil.lfc_shrink(coeff="condition_B_vs_A")
    stat_res_deconveil.summary()

    # Save results
    results_path = os.path.join(output_path, f"{replicate_num}_res_CNaware_{n_samples}_{n_genes}.csv")
    stat_res_deconveil.results_df.to_csv(results_path)
    return(stat_res_deconveil.results_df)

In [15]:
#sample_sizes = [10, 20, 40, 100]
#gene_counts = [3000]
#noise_levels = [0, 0.05, 0.10, 0.15, 0.20]

#for n_samples in sample_sizes:
    #for n_genes in gene_counts:
        #for noise in noise_levels:
            #process_and_save_single_run(n_samples, n_genes, noise)

In [15]:
sample_sizes = [10,20,40,60]
gene_counts = [5000]
num_replicates = 5
noise_levels = [0, 0.10, 0.15, 0.20, 0.25]
#noise_levels = [0.05]

In [17]:
def process_and_save_replicates(replicate_num, n_samples, n_genes, noise_level):
    try:
        noise_label = f"noise_{int(noise_level * 100)}" if noise_level > 0 else "no_noise"
        base_path = f"/Users/katsiarynadavydzenka/Documents/PhD_AI/deconveilCaseStudies/simulations/data/simulations_4/{noise_label}/"

        # Updated filenames to match your actual format
        rna_path = os.path.join(base_path, f"{replicate_num}_{n_samples}_{n_genes}_rna.csv")
        cnv_path = os.path.join(base_path, f"{replicate_num}_{n_samples}_{n_genes}_cn.csv")
        metadata_path = os.path.join(base_path, f"{replicate_num}_{n_samples}_{n_genes}_metadata.csv")

        # Load data
        rna = pd.read_csv(rna_path, index_col=0).T
        cnv = pd.read_csv(cnv_path, index_col=0).T
        cnv = (cnv * 2).astype(int)
        metadata = pd.read_csv(metadata_path, index_col=0)

        # Output directories
        output_base = "/Users/katsiarynadavydzenka/Documents/PhD_AI/deconveilCaseStudies/simulations/results/simulation_4"
        pydeseq_output_path = os.path.join(output_base, f"{noise_label}/replicates_pydeseq/")
        deconveil_output_path = os.path.join(output_base, f"{noise_label}/replicates_deconveil/")
        os.makedirs(pydeseq_output_path, exist_ok=True)
        os.makedirs(deconveil_output_path, exist_ok=True)

        # Run tools
        res_pydeseq = run_pydeseq2(rna, metadata, pydeseq_output_path)
        res_deconveil = run_deconveil(rna, metadata, cnv, deconveil_output_path)

        print(f"✅ Processed replicate {replicate_num} | Samples: {n_samples}, Genes: {n_genes}, Noise: {noise_label}")
    except Exception as e:
        print(f"Error in replicate {replicate_num} | Samples: {n_samples}, Genes: {n_genes}, Noise: {noise_label}: {e}")

In [21]:
for n_samples in sample_sizes:
    for n_genes in gene_counts:
        for noise in noise_levels:
            for replicate_num in range(1, num_replicates + 1):
                process_and_save_replicates(replicate_num, n_samples, n_genes, noise)

Fitting size factors...
... done in 0.00 seconds.

Fitting dispersions...
... done in 1.03 seconds.

Fitting dispersion trend curve...
... done in 0.15 seconds.

Fitting MAP dispersions...
... done in 1.12 seconds.

Fitting LFCs...
... done in 0.74 seconds.

Calculating cook's distance...
... done in 0.01 seconds.

Replacing 164 outlier genes.

Fitting dispersions...
... done in 0.08 seconds.

Fitting MAP dispersions...
... done in 0.07 seconds.

Fitting LFCs...
... done in 0.07 seconds.

Running Wald tests...
... done in 0.20 seconds.

Fitting MAP LFCs...
... done in 0.84 seconds.



Log2 fold change & Wald test p-value: condition B vs A
               baseMean  log2FoldChange     lfcSE      stat        pvalue  \
WLS         5191.531614       -0.773686  0.340739 -2.627201  8.609035e-03   
AF111169.3    12.003791       -0.793748  0.514606 -2.014145  4.399428e-02   
SP140L       952.469891        0.424510  0.210699  2.140915  3.228090e-02   
AL592295.5    31.476133        0.079543  0.295400  0.302572  7.622164e-01   
DNAAF5      2196.633415        0.977199  0.161013  6.221182  4.934241e-10   
...                 ...             ...       ...       ...           ...   
FUNDC2      3568.739816        0.169095  0.150746  1.104331  2.694494e-01   
SF3A3       4659.667793       -0.023429  0.128156 -0.151320  8.797231e-01   
AC063919.1     6.308126       -5.286879  1.149025 -4.984410  6.215129e-07   
AL022328.3    18.384749       -0.826085  0.283908 -3.173063  1.508399e-03   
AL162274.2    19.839087       -0.587957  0.308023 -2.145080  3.194643e-02   

                    

Fitting size factors...
... done in 0.01 seconds.

Fitting dispersions...
... done in 0.77 seconds.

Fitting dispersion trend curve...
... done in 0.11 seconds.

Fitting MAP dispersions...
... done in 1.03 seconds.

Fitting LFCs...
... done in 0.55 seconds.

Calculating cook's distance...
... done in 0.01 seconds.

Replacing 161 outlier genes.

Fitting dispersions...
... done in 0.06 seconds.

Fitting MAP dispersions...


replace_mask before filtering: (20, 161)
Number of True values in replace_mask: 187
replacement_counts_trimmed shape: (19, 161)


... done in 0.07 seconds.

Fitting LFCs...
... done in 0.07 seconds.

Running Wald tests...
... done in 0.16 seconds.

Fitting MAP LFCs...
... done in 0.73 seconds.



...

Fitting size factors...
... done in 0.02 seconds.

Fitting dispersions...
... done in 1.01 seconds.

Fitting dispersion trend curve...
... done in 0.11 seconds.

Fitting MAP dispersions...
... done in 1.21 seconds.

Fitting LFCs...
... done in 0.58 seconds.

Calculating cook's distance...
... done in 0.07 seconds.

Replacing 1351 outlier genes.



replace_mask before filtering: (120, 1351)
Number of True values in replace_mask: 1639
replacement_counts_trimmed shape: (88, 1351)


Fitting dispersions...
... done in 0.21 seconds.

Fitting MAP dispersions...
... done in 0.40 seconds.

Fitting LFCs...
... done in 0.18 seconds.

Running Wald tests...
... done in 0.29 seconds.

Fitting MAP LFCs...


Log2 fold change & Wald test p-value: condition B vs A
               baseMean  log2FoldChange     lfcSE       stat        pvalue  \
IGLV5-37      31.160138       -1.857585  0.420590  -4.768420  1.856761e-06   
FAM13A-AS1   185.209759       -1.067814  0.180268  -6.028025  1.659754e-09   
DCTN5       4641.594332        0.591334  0.020699  -0.062156  9.504388e-01   
PELI1       3021.092798       -0.029784  0.058782 -11.313727  1.122194e-29   
LINC00639    114.451550       -1.622439  0.111374  -8.216910  2.088144e-16   
...                 ...             ...       ...        ...           ...   
CCNDBP1     3334.402768       -0.269430  0.035672  -1.837357  6.615721e-02   
TNXB        7605.029648       -3.786207  0.243556 -14.465700  1.995642e-47   
HS6ST2       361.535708       -0.385323  0.275858  -1.554276  1.201185e-01   
ZNF316      1865.267311        0.105813  0.103874  -0.808230  4.189582e-01   
SPATA12       16.906896        1.414984  0.225601   5.935026  2.937997e-09   

        

... done in 0.89 seconds.

